<a href="https://colab.research.google.com/github/SeoJiWon1/ML-DL/blob/main/Pytorch_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN으로 MNIST 분류하기

* 합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 조금 다르다
* 1. 첫번째 표기 방법

합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명

* 2. 두번째 표기 방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층

편의를 위해 맥스풀링까지도 포함해서 하나의 합성곱 층으로 판단하고 정리

## 모델의 아키텍처: 총 3개의 층으로 구성

### 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))


### 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))



### 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax

## 모델 구현


### 1. 필요한 도구 임포트와 입력 정의.

In [1]:
import torch
import torch.nn as nn

In [2]:
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1,1,28,28)
print(f'텐서의 크기:{inputs.shape}')

텐서의 크기:torch.Size([1, 1, 28, 28])


### 2. 합성곱층과 풀링 선언하기

* 첫번째 합성곱 층을 구현
* 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1

In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding =1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


* 두 번째 합성곱 층을 구현
* 32 채널 입력 64채널을 뽑아내는데 커널 사이즈는 3 패딩은 1

In [4]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding =1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


* 맥스풀링 구현현

In [5]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


### 3. 구현체를 연결하여 모델 만들기
* 지금까지는 선언만 연결은x 
* 이들을 연결시켜서 모델을 완성
* 첫 번째 합성곱층 통과
* 통과시킨 후 텐서의 크기 비교

In [6]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널의 28너비 28높이의 텐서

32가 나온 이유는 conv1의 out_channel로 32를 지정해주었기 때문

또한, 28너비 28높이가 된 이유는 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존

* 맥스풀링을 통과시키고 통과한 후의 텐서 크기 비교

In [7]:
out  = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [8]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


* 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 비교

In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


*  .view()를 사용하여 텐서를 펼치는 작업

In [10]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [11]:
# n번째 차원을 접근하게 해주는 .size(n)
# 현재 out의 크기 1 x 64 x 7 x 7
# out의 첫 번째 차원이 몇인지 출력
out.size(0)


1

In [12]:
out.size(1)

3136

* 이제 이에 대해서 전결합층(Fully-Connteced layer)를 통과
* CNN은 합성곱층, 풀링층, 전결합층으로 구성
* 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환

In [13]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim =10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## CNN으로 MNIST 분류하기

### (1) 필요한 도구 임포트

In [17]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

### (2) GPU 사용 가능 여부

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


### (3) 학습에 사용할 파라미터 설정

In [30]:
learning_rate = 0.001
training_epochs = 15
batch_size  = 100

### (4) 데이터로더를 사용하여 데이터셋을 정의

In [31]:
mnist_train = dsets.MNIST(root = "MNIST_data/",
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root = "MNIST_data/",
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)


### (5) 데이터로더를 사용하여 배치 크기를 지정

In [32]:
data_loader = torch.utils.data.DataLoader(dataset =mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

### (6) 클래스로 모델을 설계

In [33]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out


## 모델을 정의

In [34]:
# CNN 모델 정의
model = CNN().to(device)

## 비용 함수와 옵티마이저를 정의

In [35]:
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수가 포함되어져 있음
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

## 총 배치의 수 출력

In [36]:
total_batch = len(data_loader)
print(f"총 배치의 수 : {total_batch}")

총 배치의 수 : 600


* 총 배치의 수는 600

* 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개란 의미

## 모델을 훈련

In [37]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.229303986
[Epoch:    2] cost = 0.0645529777
[Epoch:    3] cost = 0.0467760526
[Epoch:    4] cost = 0.0383122563
[Epoch:    5] cost = 0.0311420057
[Epoch:    6] cost = 0.0265621524
[Epoch:    7] cost = 0.0218592118
[Epoch:    8] cost = 0.0190796815
[Epoch:    9] cost = 0.0163430367
[Epoch:   10] cost = 0.0140317129
[Epoch:   11] cost = 0.0123439338
[Epoch:   12] cost = 0.010240824
[Epoch:   13] cost = 0.00755288964
[Epoch:   14] cost = 0.00907103904
[Epoch:   15] cost = 0.00620678533


## 테스트

In [38]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9872999787330627


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
